In [1]:
import torch

# Storage and View
1. 每个tensor都有对应的底层data放在memory上
   - 用tensor.storage()查看底层data的信息
   - tensor.data_ptr()返回tensor第一个元素的地址.可以用a.storage().data_ptr() == b.storage().data_ptr()来判断两个tensor底层数据是不是同一个。<font color=red>注意，这里只能用'=='，不能用'is'，因为这两个pointer本身是两个不同的pointer，只是他们的值相同</font>
2. tensor有两种layout attribute，对应两种不同类型的tensor：dense和sparse tensor。
   - dense tensor的底层data占用连续的存储空间，对应layout attribute是torch.stride。它用stride对象提供信息，决定如何从连续的data sequence上读取tensor element。
   - sparse tensor则占用非连续空间，对应的layout attribute是torch.sparse。略<font color=red>[这里只关注dense tensor]</font>
3. 每个dense tensor基于它的底层data有自己呈现data的方式，也就是view。不同的tensor可以共享相同的data memory，但是view不同。
4. **常见的只改变view，不copy data的reshape method**：这些是可以通过改变stride信息来得到新view的method，不需要copy data。
   1. <font color=blue>**tensor.view()**</font>：dense tensor如果是continuous tensor，可以用tensor.view() method改变tensor的view形态，得到的新tensor的data sequence的排序和原数据在memory中的排序一样，即不改变原tensor的continuity。<font color=green>如果要得到一个新shape的tensor，但是不知道是否能保证keep continuity，最好改用tensor.reshape()</font>
   2. <font color=blue>**tensor.transpose()和tensor.movedim(source, destination)**</font>：
      - 他们都是两种dense tensor(continuous和non-continuous)都可以用，得到的tensor的data sequence的排序和原数据在memory中的排序不同，因此此时continuity会被改变。
      - transpose只能两个维度对调位置
      - movedim更灵活，可以同时调整多个维度
   3. <font color=blue>**tensor.squeeze(), tensor.unsqueeze()和tensor.expand()**</font>: 两种dense tensor(continuous和non-continuous)都可以用。
      - <font color=blue>**tensor.expand()**</font>是扩展dim length=1的维度，扩展是指给重复的多个view。可以增加一个维度，相当于先做unsqueeze在dim0加一个dim length=1的维度，再扩展。
   4. <font color=blue>**tensor.vsplit(), tensor.hsplit(), tensor.split(), tensor.chunk() and tensor.unflatten()**</font>
      - hsplit和vsplit中，如果参数是int，要求对应dim的length要整除该int
      - split最灵活，如果给的参数是int，对应dim的length如果不能整除，那么最后一个分出来的section的数量最少
      - 三种\*split method都可以通过设置参数为list，把原tensor切成大小不同的chunk，但是chunk method只能做平分，只是切出来最后一个chunk大小可能小一点。
      - tensor.unflatten()不是切割，而是把指定维度的数据转成指定的shape。<font color=red>注，tensor.flatten()可能copy data</font>

## 1. tensor的存储方式和特点

### 1.1 tensor.storage()
输出tensor存在memory中的1D data sequence

In [2]:
x = torch.tensor([[1, 2, 3], [6, 7, 8]])
x.storage(), x.is_contiguous()

/tmp/ipykernel_26087/3405978919.py:2: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  x.storage(), x.is_contiguous()


( 1
  2
  3
  6
  7
  8
 [torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 6],
 True)

### 1.2 dense和sparse tensor有不同的layout attribute
1. 根据存储方式的不同，tensor有两种类型，dense tensor和sparse tensor，他们可以通过tensor.layout attribute来区分\
(1)**dense tensor** is stored linearly in a contiguous block of memory，也就是其存储占用了连续的memory block。\
(2)**sparse tensor**中大部分是0，只有很少比例的值是非零值，为了提高存储效率，pytorch为这类数据提供了特殊的存储方式。
2. torch.layout attribute是表达tensor的memory layout特征的object。这个attribute有两种类型的取值：torch.stride和torch.sparse_coo，分别对应上面的两种tensor类型\
(1)如果<font color=blue>**layout=torch.strided**</font>，那么该tensor是dense tensor\
(2)如果<font color=blue>**layout=torch.sparse**</font>，那么该tensor是sparse tensor

In [3]:
## 新建dense tensor，得到连续的存储空间
#  连续的存储空间是指底层的data，改变view后的新tensor的元素可能不连续
x = torch.rand((2, 3), layout=torch.strided, requires_grad=False)

# 这里y是x做transpose后的View，这个view的element相对实际存储的data而言不连续
y = x.T
x.is_contiguous(), x.stride(), y.is_contiguous(), y.stride()

(True, (3, 1), False, (1, 3))

In [4]:
## 新建sparse tensor，存储空间本身就不连续
z = torch.zeros((100, 200), layout=torch.sparse_coo, requires_grad=False)
z, z.is_contiguous()

(tensor(indices=tensor([], size=(2, 0)),
        values=tensor([], size=(0,)),
        size=(100, 200), nnz=0, layout=torch.sparse_coo),
 False)

### 1.3 tensor.stride()可以查看dense tensor读取底层data的方式
1. dense tensor才有**tensor.stride()** method
   - dense tensor不论是contiguous还是non-contiguous tensor都有stride。
2. strided tensor的stride是一个int list,其第k-th元素的值表示tensor的k-th dimension上，一个元素到下个元素之间，用units of elements衡量的长度。
3. 每个strided tensor都有一个与之关联的torch.Storage存放data. 这些tensors 则提供strided view of a storage。一个torch.Storage可以对应不同的strided tensors。这些tensors有相同的data，不同的tensor.view()
   - <font color=blue>**Numpy strides()** returns (N bytes to Next Row, M bytes to Next Column)
   - **Pytorch stride()** returns (N elements to Next Row, M elements to Next Column).</font>

In [5]:
x = torch.tensor([[1, 2, 3, 4, 5],
                  [6, 7, 8, 9, 10]])

print(x.stride())  # 不指定dim，A tuple of all strides is returned
                   # 在tuple(5, 1)中，5在第0维，1在第1维对应traverse的维度
print(x.stride(0)) # 指定dim=0，返回该维度上的stride值5
                   # traverse along the 0th dim, 比如从'1'跳到'6',距离是5
print(x.stride(-1))# 指定dim=1，返回该维度上的stride值1
                   # traverse along the 1st dim, 比如从'7'跳到'8',距离是1

## 改变view，stride也改变
y = x.view(5, 2)
print(y.stride())

(5, 1)
5
1
(2, 1)


### 1.4 dense tensor的底层data存储在连续位置

1. dense tensor可能是contiguous tensor或者non-contiguous tensor。虽然他们在存储空间中都占用了连续的存储单元，但non-contiguous tensor的元素排列顺序与其元素在存储空间的排列顺序不一致。\
**· <font color=blue>tensor.view()</font>** 会保持contiguous tensor的contiguity。也只有contiguous tensor有view() method\
**· <font color=blue>tensor.transpose()</font>** 会改变tensor的contiguity。contiguous和non-contiguous tensor都可以transpose。当base tensor是dense tensor时，transpose不copy data，所以output tensor也只是原underlying data的一个new view。
2. 用is_contiguous()可以查看是否contiguous
3. 用tensor.contiguous()可以把non-contiguous tensor转变成contiguous

In [6]:
b = torch.tensor([[0, 1],[2, 3]])
t = b.transpose(0, 1) 
print(t, b.data_ptr() == t.data_ptr()) # 同一data
b.is_contiguous(), t.is_contiguous() # continuity变化

tensor([[0, 2],
        [1, 3]]) True


(True, False)

In [7]:
## 用tensor.storage()可以获得该tensor在memory上的1D sequence
x = torch.arange(0,6)

## view返回的tensor中，data sequence的排序和原数据相同，都是0, 1, 2, 3, 4...
y = x.view(2,3)
y.stride(), y.is_contiguous()

((3, 1), True)

In [8]:
y.storage().data_ptr() == x.storage().data_ptr()

True

In [9]:
## transpose返回的tensor中，data sequence的排序和原数据在memory中的排序不同
#  z中元素的排序是：0, 3, 1, 4...
#  transpose不copy data，它仍然是原data的view，但是是non-contiguous ‘View’
#  按新tensor的element顺序从memory中取值的时候不是连续取值
z = y.T
print('z: \n', z)
print(z.stride(), z.is_contiguous(), y.storage().data_ptr() == z.storage().data_ptr())

z: 
 tensor([[0, 3],
        [1, 4],
        [2, 5]])
(1, 3) False True


In [10]:
## transpose将non-contiguous转变成contiguous
w = z.T
w.is_contiguous()

True

In [11]:
## 用tensor.contiguous()转变成contiguous tensor
#  此时发生了copy，按照tensor view的排序方式复制了一个新的data
v = z.contiguous()
print('v: \n', v, v.is_contiguous())
print(v.storage().data_ptr() == z.storage().data_ptr())

v: 
 tensor([[0, 3],
        [1, 4],
        [2, 5]]) True
False


## 2. 通过改变stride改变view的method
### 2.1 用tensor.view()不改变torch.layout属性
<font color=red>相当于numpy里面的reshape。</font>
1. 只有contiguous dense tensor才有view method。
2. dense tensor在计算机中以1D data sequence的形式存在congtiguous block of memory上。tensor.view()本质上是告诉计算机如何(根据其参数设置)stride over the 1D data sequence。所以，只提供了new view of the base tensor，不改变底层data。view的改变也就意味着tensor.stride()也会随之改变。
3. tensor.view()返回的new tensor从memory上读取elements的顺序与其线性存储的顺序一致，<font color=red>只是改变了划分layer,column和row等unit单位的位置。</font>
4. 因为View tensor与它的base tensor的underlying data是同一个data，即tensor.view()返回new tensor，但并不copy data，所以，其中一个tensor元素值改变，另一个也变。

<font color=blue>**tensor.data_ptr()**</font>: 返回1st element的存储地址，即data pointer指向的值

In [12]:
## tensor.view()返回的新tensor不copy data，但是会改变stride和shape(size)
a = torch.arange(15)
print('原data pointer:', a.data_ptr()) # 
print('原id:', id(a))
print('原stride和size:', a.stride(), a.size(), '\n')

# change view
b = a.view(3, 5)
print('新data pointer与原值相同:', b.data_ptr())
print('id变化，说明新建了一个tensor:', id(b))                 # id变化
print('stride和size变化:', b.stride(), b.size()) 

# data地址不变
a.data_ptr() == b.data_ptr()

原data pointer: 139062656
原id: 140668228855552
原stride和size: (1,) torch.Size([15]) 

新data pointer与原值相同: 139062656
id变化，说明新建了一个tensor: 140668228856032
stride和size变化: (5, 1) torch.Size([3, 5])


True

In [13]:
## tensor.view()不改变tensor layout in memory
a = torch.rand(12)
b = a.view(3, 2, 2)
c = a.view(2, 3, 2)  # 不改变tensor layout in memory

print('a: \n', a)
print('b: \n', b) # b的元素排序和a完全相同，只是每个dim的截断位置改变

print(id(a), id(b), id(c)) # id变化, 是不同的tensor

a: 
 tensor([0.5790, 0.9373, 0.9461, 0.0924, 0.1483, 0.3489, 0.3139, 0.5684, 0.3756,
        0.8025, 0.0997, 0.8067])
b: 
 tensor([[[0.5790, 0.9373],
         [0.9461, 0.0924]],

        [[0.1483, 0.3489],
         [0.3139, 0.5684]],

        [[0.3756, 0.8025],
         [0.0997, 0.8067]]])
140668228850672 140668228855952 140668228856032


### 2.2 用transpose和movedim会改变contiguity
<font color=red>注：这里的view的layout是指view的shape，不是tensor的layout属性。</font>

#### 2.2.1 tensor.transpose()
1. 和view method不同的是，transpose改变了elements在view中的layout。含义是：如果把transpose之后的tensor排成一个vector，其顺序与input tensor不相同。
2. 和view method相同的是，当tensor是dense tensor时，output只是input的一个view，不发生copy。改变其中一个的元素值，另一个也变
3. <font color=blue>**tensor.adjoint(),tensor.mT和tensor.mH**</font>在tensor元素为实数的时候相当于tensor.transpose(-2,-1)。返回原tensor交换最后两个维度后的view。mT和mH的区别在于处理复数的时候一个可以实现和adjoin一样的功能，一个不能。

In [14]:
a = torch.arange(24).reshape(1, 2, 3, 4)
print("原shape:", '\n', a)
print(a.size(), id(a), '\n')

原shape: 
 tensor([[[[ 0,  1,  2,  3],
          [ 4,  5,  6,  7],
          [ 8,  9, 10, 11]],

         [[12, 13, 14, 15],
          [16, 17, 18, 19],
          [20, 21, 22, 23]]]])
torch.Size([1, 2, 3, 4]) 140668228856832 



In [15]:
b = a.transpose(1, 2)
print("用transpose交换了2nd和3rd dim:", '\n', b)
print(b.size(), id(b), '\n')

a.view(2, 12)
# b.view(24) # 错，b这时不是contiguous，不能用view method

用transpose交换了2nd和3rd dim: 
 tensor([[[[ 0,  1,  2,  3],
          [12, 13, 14, 15]],

         [[ 4,  5,  6,  7],
          [16, 17, 18, 19]],

         [[ 8,  9, 10, 11],
          [20, 21, 22, 23]]]])
torch.Size([1, 3, 2, 4]) 140668228858672 



tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11],
        [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]])

In [16]:
c = a.mT # 注意mT不是函数，mutate tail
d = a.mH # 注意mH不是函数，mutate Head

c, c.shape, torch.all(c == d)

(tensor([[[[ 0,  4,  8],
           [ 1,  5,  9],
           [ 2,  6, 10],
           [ 3,  7, 11]],
 
          [[12, 16, 20],
           [13, 17, 21],
           [14, 18, 22],
           [15, 19, 23]]]]),
 torch.Size([1, 2, 4, 3]),
 tensor(True))

#### 2.2.2 movedim(source, destination)
和transpose一样，它会改变tensor的view的layout，得到的是non-contiguous tensor，也就是不能按照原data sequence来从底层data读取element。但是不会copy data。

In [17]:
t = torch.arange(24).view(2, 3, 4)
t

tensor([[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11]],

        [[12, 13, 14, 15],
         [16, 17, 18, 19],
         [20, 21, 22, 23]]])

In [18]:
## 当参数是int的时候，int代表dim，
t.movedim(2, 0) # 把dim2移动到dim0的位置

tensor([[[ 0,  4,  8],
         [12, 16, 20]],

        [[ 1,  5,  9],
         [13, 17, 21]],

        [[ 2,  6, 10],
         [14, 18, 22]],

        [[ 3,  7, 11],
         [15, 19, 23]]])

In [19]:
## 当参数是tuple的时候，分别代表source和destination dims
s = t.movedim((1, 2), (0, 1)) # 把dim1移动到dim0，再把dim2移动到dim1
s.shape, s

(torch.Size([3, 4, 2]),
 tensor([[[ 0, 12],
          [ 1, 13],
          [ 2, 14],
          [ 3, 15]],
 
         [[ 4, 16],
          [ 5, 17],
          [ 6, 18],
          [ 7, 19]],
 
         [[ 8, 20],
          [ 9, 21],
          [10, 22],
          [11, 23]]]))

### 2.3 squeeze and unsqueeze
 - squeeze和unsqueeze只改变原数据的view，新生成的tensor与input tensor指向相同的memory
#### 2.3.1 tensor.unsqueeze()
1. pytorch默认是按照batch处理数据，以image为例，默认的input大小是(N, C, H, W)。如果想让model处理单个数据，就要把当个样本的大小从(C, H, W)改成(1, C, H, W)
2. 常用于ease broadcast

In [20]:
# 用tensor.unsqueeze()来增加长度为1的新维度
a = torch.rand(3, 226, 226)
b = a.unsqueeze(0) # 在第0维增加一个维度
print('增加一个长度为1的维度到新的第0维：',b.shape)

c = a.unsqueeze(1)
print('增加一个长度为1的维度到新的第1维：',c.shape)

增加一个长度为1的维度到新的第0维： torch.Size([1, 3, 226, 226])
增加一个长度为1的维度到新的第1维： torch.Size([3, 1, 226, 226])


In [21]:
# unsqueeze常用于方便broadcast
a = torch.ones(4, 3, 2)
b = torch.arange(3)     # a * b不能直接运算
print('shape before:', b.shape)
c = b.unsqueeze(1)       # change to a 2-dimensional tensor, adding new dim at the end
print('shape after:', c.shape)
print(a * c)             # broadcasting works again!

shape before: torch.Size([3])
shape after: torch.Size([3, 1])
tensor([[[0., 0.],
         [1., 1.],
         [2., 2.]],

        [[0., 0.],
         [1., 1.],
         [2., 2.]],

        [[0., 0.],
         [1., 1.],
         [2., 2.]],

        [[0., 0.],
         [1., 1.],
         [2., 2.]]])


#### 2.3.2 tensor.squeeze()
1. 用tensor.squeeze()来压缩长度为1的维度(dimensions of extent 1)
2. 最好指定dim number，因为pytorch按batch处理数据，squeeze()会把第1维N压掉，导致模型出错 

In [22]:
a = torch.ones(1, 2, 1, 3)
b = a.squeeze()   # 压缩所有维度为1的dims
c = a.squeeze(0)  # 压缩第0维
print(b.shape)
print(c.shape)

## 改变b之后，a的值也变
b += 1
a, a.stride()

torch.Size([2, 3])
torch.Size([2, 1, 3])


(tensor([[[[2., 2., 2.]],
 
          [[2., 2., 2.]]]]),
 (6, 3, 3, 1))

#### 2.3.3 tensor.expand(newshape)
- 以repeat的方式扩展dim length=1的维度
- 不扩展的维度可以在参数中的对应dim上用-1
- 可以增加一维到dim0，假如原来的shape是(a, b...)，这里参数设为(n, a, b...)

In [23]:
x = torch.tensor([[[1,2], [3,4], [5,6]]])
x.size()

torch.Size([1, 3, 2])

In [24]:
# 以repeat的方式扩展dim length=1的维度
x.expand(2, 3, 2).shape, x.expand(2, 3, 2)

(torch.Size([2, 3, 2]),
 tensor([[[1, 2],
          [3, 4],
          [5, 6]],
 
         [[1, 2],
          [3, 4],
          [5, 6]]]))

In [25]:
## -1表示不改变对应维度
torch.all(x.expand(2, 3, 2) == x.expand(2, -1, -1))  

tensor(True)

In [26]:
## 可以增加一维到dim0
x.expand(2, 1, 3, 2)

tensor([[[[1, 2],
          [3, 4],
          [5, 6]]],


        [[[1, 2],
          [3, 4],
          [5, 6]]]])

### 2.4 tensor.split(), tensor.chunk() and tensor.unflatten()
1. split不同于numpy中的array_split，主要是参数的含义不同
   - int参数
     - tensor.split中指定的是每份的size是多大
     - numpy.array_split中指定要分多少份，和tensor.chunk一样
   - list或者tuple参数
     - tensor.split中指定的是每份的length数量
     - numpy.array_split中是给的分割点
2. chunk灵活性不如split，只能做"平分"，不能分成指定的数量不均匀的chunk。
   - 分割得到n份，前l % n份的size是l // n + 1，后面的是l // n。<font color=red>注，这里分法有点奇怪，是先每份给l // n,再把余数l % n分给前l % n份。</font>
3. unflatten把指定维度改成新的shape。在指定dim上重新改shape的时候不需要改变原tensor的element sequence

In [27]:
## tensor.split
a = torch.arange(10).reshape(5, 2)
a, torch.split(a, [1, 4]) # tuple中元素sum要和a中dim0的length一样，不然报错

(tensor([[0, 1],
         [2, 3],
         [4, 5],
         [6, 7],
         [8, 9]]),
 (tensor([[0, 1]]),
  tensor([[2, 3],
          [4, 5],
          [6, 7],
          [8, 9]])))

In [28]:
## tensor.chunk(int, dim=0): int指的是dim要分成多少份
print('chunk\n case1: ', torch.arange(7).chunk(4))
print('case2: ', torch.arange(9).chunk(4)) # 得到的chunk数量可能不等于参数

#  对比split
print('split\n case1: ', torch.arange(7).split(4))
print('case2: ', torch.arange(9).split(4))

chunk
 case1:  (tensor([0, 1]), tensor([2, 3]), tensor([4, 5]), tensor([6]))
case2:  (tensor([0, 1, 2]), tensor([3, 4, 5]), tensor([6, 7, 8]))
split
 case1:  (tensor([0, 1, 2, 3]), tensor([4, 5, 6]))
case2:  (tensor([0, 1, 2, 3]), tensor([4, 5, 6, 7]), tensor([8]))


In [29]:
## unflatten把指定维度改成新的shape
c = torch.arange(16).view(2, 4, 2)
new_c = torch.unflatten(c, 1, (2, 2, 1)) # 把c的dim1转变成shape:(2, 2)
c, new_c.shape, new_c

(tensor([[[ 0,  1],
          [ 2,  3],
          [ 4,  5],
          [ 6,  7]],
 
         [[ 8,  9],
          [10, 11],
          [12, 13],
          [14, 15]]]),
 torch.Size([2, 2, 2, 1, 2]),
 tensor([[[[[ 0,  1]],
 
           [[ 2,  3]]],
 
 
          [[[ 4,  5]],
 
           [[ 6,  7]]]],
 
 
 
         [[[[ 8,  9]],
 
           [[10, 11]]],
 
 
          [[[12, 13]],
 
           [[14, 15]]]]]))